## 1. Prepare our data

<font color='blue'>1.1 Import necessary libraries</font>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.0                |           py37_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0

The followi

<font color='blue'>1.2 Scrape the neiborhood list from Wikipidia</font>

In [2]:
from bs4 import BeautifulSoup
URL = "https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

<font color='blue'>1.3 Build our neiborhood dataframe</font>

In [4]:
neib=list()
for items in soup.find_all('li', class_='toclevel-1'):
    data = items.find_all('span', class_="toctext")
    try:
        if data[0].text=='See also':
            break
        else:
            aa=data[0].text
        neib.append(aa)
    except IndexError:pass
dfo=pd.DataFrame(neib,columns=['Neighborhood'])
print('The shape of the dataframe neib is ',dfo.shape)
dfo.head()

The shape of the dataframe neib is  (119, 1)


,Neighborhood
0,Alamo Square
1,Anza Vista
2,Ashbury Heights
3,Balboa Park
4,Balboa Terrace


<font color='blue'>1.3 Add latitude and longitude coordinates info into dataframe</font>

In [5]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ny_explorer")

In [6]:
add=dfo+', San Francisco, CA, USA'
add.columns=['Address']
add.head()

,Address
0,"Alamo Square, San Francisco, CA, USA"
1,"Anza Vista, San Francisco, CA, USA"
2,"Ashbury Heights, San Francisco, CA, USA"
3,"Balboa Park, San Francisco, CA, USA"
4,"Balboa Terrace, San Francisco, CA, USA"


In [7]:
#Find the 
dist=list()
lati=list()
logi=list()
for address in add['Address']:
    try:
#        print(address)
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        dist.append(address.split(',')[0])
        lati.append(latitude)
        logi.append(longitude)
#        print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))
    except AttributeError:pass

In [8]:
df=pd.DataFrame(list(zip(dist,lati,logi)),columns=['Neighborhood','Latitude','Longitude'])
print('The shape of dataframe df is', df.shape)
df.head()

The shape of dataframe df is (90, 3)


,Neighborhood,Latitude,Longitude
0,Alamo Square,37.776360,-122.434689
1,Anza Vista,37.780836,-122.443149
2,Balboa Park,37.724949,-122.444805
3,Bayview,37.728889,-122.392500
4,Belden Place,37.791744,-122.403886


## 2. Create a map of San Francisco with neiborhoods superimposed

<font color='blue'>2.1 Get the geograpical coordinate of San Francisco</font>

In [9]:
address_s = 'San Francisco, CA, USA'

geolocator_s = Nominatim(user_agent="ny_explorer")
location_s = geolocator.geocode(address_s)
latitude_s = location_s.latitude
longitude_s = location_s.longitude
print(location_s)
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude_s, longitude_s))

San Francisco, San Francisco City and County, California, United States of America
The geograpical coordinate of San Francisco are 37.7792808, -122.4192363.


<font color='blue'>2.2 Create the map and add neiborhoods</font>

In [10]:
map_sf = folium.Map(location=[latitude_s, longitude_s], zoom_start=12)
map_sf

In [11]:
# add markers to map
for neibo, lat, lng in zip(df['Neighborhood'],df['Latitude'], df['Longitude']):
    label = '{}, {}'.format(neibo, 'San Francisco')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)  
    
map_sf

<font color='blue'>2.3 Create the map with the neiborhoods that we are more concerned.</font>

In [12]:
#Select the neighborhoods that near to the downtown.
df1=df.loc[(df['Latitude'] >= 37.760956) & (df['Latitude'] <= 37.809167)]
dfn=df1.loc[(df1['Longitude']>=-122.456638)&(df1['Longitude']<=-122.386033)]
dfn.shape

(45, 3)

In [13]:
#View the map with neighborhoods in downtown.
map_sf = folium.Map(location=[latitude_s, longitude_s], zoom_start=12.5)
for neibo, lat, lng in zip(dfn['Neighborhood'],dfn['Latitude'], dfn['Longitude']):
    label = '{}, {}'.format(neibo, 'San Francisco')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)  
    
map_sf

## 3. Use Foursquare API to explore the neighborhoods of San Francisco

<font color='blue'>3.1 Define Foursquare Credentials and Version.</font>

In [38]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


<font color='blue'>3.2 Define a function which can output the venues' information.</font>

In [17]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
#Load the venues data into sf_venues
sf_venues = getNearbyVenues(names=dfn['Neighborhood'],
                                   latitudes=dfn['Latitude'],
                                   longitudes=dfn['Longitude']
                                  )

<font color='blue'>3.3 Check the size of resulting dataframe.</font>

In [19]:
vinfo=pd.DataFrame(sf_venues)
print('The shape of vinfo is ',vinfo.shape)
vinfo.head()

The shape of vinfo is  (3428, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alamo Square,37.77636,-122.434689,Alamo Square,37.776143,-122.434269,Park
1,Alamo Square,37.77636,-122.434689,Alamo Square Dog Park,37.775878,-122.435740,Dog Run
2,Alamo Square,37.77636,-122.434689,Painted Ladies,37.776120,-122.433389,Historic Site
3,Alamo Square,37.77636,-122.434689,The Independent,37.775573,-122.437835,Rock Club
4,Alamo Square,37.77636,-122.434689,The Mill,37.776425,-122.437970,Bakery


In [21]:
vinfo.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alamo Square,73,73,73,73,73,73
Anza Vista,21,21,21,21,21,21
Belden Place,100,100,100,100,100,100
Buena Vista,61,61,61,61,61,61
China Basin,85,85,85,85,85,85
Chinatown,100,100,100,100,100,100
Civic Center,95,95,95,95,95,95
Cole Valley,53,53,53,53,53,53
Corona Heights,35,35,35,35,35,35


In [22]:
print('There are {} uniques categories.'.format(len(vinfo['Venue Category'].unique())))

There are 305 uniques categories.


## 4. Analyze Each neighborhood

In [23]:
# one hot encoding
sf_onehot = pd.get_dummies(vinfo[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sf_onehot['Neighborhood'] = vinfo['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]
print('The shape of sf_onehot is ',sf_onehot.shape)
sf_onehot.head()

The shape of sf_onehot is  (3428, 305)


,Yoga Studio,Acai House,Accessories Store,Afghan Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Eye Doctor,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Physical Therapist,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaur

In [25]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
print('The shape of sf_grouped is ',sf_grouped.shape)
sf_grouped.head()

The shape of sf_grouped is  (45, 305)


,Neighborhood,Yoga Studio,Acai House,Accessories Store,Afghan Restaurant,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Eye Doctor,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flower Shop,Food,Food & Drink Shop,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Physical Therapist,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaur

In [26]:
#Try 5
num_top_venues = 5

for hood in sf_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sf_grouped[sf_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alamo Square----
              venue  freq
0               Bar  0.05
1       Pizza Place  0.03
2  Sushi Restaurant  0.03
3             Hotel  0.03
4       Record Shop  0.03


----Anza Vista----
                     venue  freq
0                     Café  0.14
1  Health & Beauty Service  0.10
2                Juice Bar  0.05
3        Convenience Store  0.05
4                 Bus Line  0.05


----Belden Place----
                  venue  freq
0           Coffee Shop  0.10
1                   Gym  0.04
2                  Café  0.03
3            Food Truck  0.03
4  Gym / Fitness Center  0.03


----Buena Vista----
                venue  freq
0      Ice Cream Shop  0.05
1  Seafood Restaurant  0.05
2       Historic Site  0.05
3                Park  0.05
4     Harbor / Marina  0.03


----China Basin----
                     venue  freq
0         Baseball Stadium  0.11
1              Coffee Shop  0.08
2  New American Restaurant  0.05
3     Gym / Fitness Center  0.04
4          Thai Restaura

                   venue  freq
0           Cocktail Bar  0.04
1               Beer Bar  0.03
2            Coffee Shop  0.03
3                Theater  0.03
4  Performing Arts Venue  0.03


----Western Addition----
                venue  freq
0        Liquor Store  0.08
1            Boutique  0.08
2       Grocery Store  0.08
3  Seafood Restaurant  0.04
4         Pizza Place  0.04




In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
#Try 10
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Square,Bar,Hotel,Liquor Store,Seafood Restaurant,Café,Sushi Restaurant,Record Shop,Pizza Place,Ethiopian Restaurant,Indian Restaurant
1,Anza Vista,Café,Health & Beauty Service,Bus Stop,Bus Line,Coffee Shop,Tunnel,Grocery Store,Arts & Crafts Store,Big Box Store,Donut Shop
2,Belden Place,Coffee Shop,Gym,Cocktail Bar,Gym / Fitness Center,Café,Sushi Restaurant,Bubble Tea Shop,Food Truck,Men's Store,Boutique
3,Buena Vista,Park,Seafood Restaurant,Historic Site,Ice Cream Shop,Harbor / Marina,Chocolate Shop,Diner,Gift Shop,Clothing Store,Boat or Ferry
4,China Basin,Baseball Stadium,Coffee Shop,New American Restaurant,Gym / Fitness Center,Baseball Field,Bar,Outdoor Sculpture,Pier,Pizza Place,Athletics & Sports


## 5. Cluster neighborhoods

<font color='blue'>5.1 Run K-means to cluster neighborhoods into 5 clusters.</font>

In [29]:
kclusters = 5

sf_grouped_clustering = sf_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 1, 2, 0, 1, 1, 1, 1], dtype=int32)

<font color='blue'>5.2 Run K-means to cluster neighborhoods into 10 clusters.</font>

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sf_merged = dfn

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sf_merged = sf_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

sf_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Square,37.776360,-122.434689,1,Bar,Hotel,Liquor Store,Seafood Restaurant,Café,Sushi Restaurant,Record Shop,Pizza Place,Ethiopian Restaurant,Indian Restaurant
1,Anza Vista,37.780836,-122.443149,2,Café,Health & Beauty Service,Bus Stop,Bus Line,Coffee Shop,Tunnel,Grocery Store,Arts & Crafts Store,Big Box Store,Donut Shop
4,Belden Place,37.791744,-122.403886,2,Coffee Shop,Gym,Cocktail Bar,Gym / Fitness Center,Café,Sushi Restaurant,Bubble Tea Shop,Food Truck,Men's Store,Boutique
6,Buena Vista,37.806532,-122.420648,1,Park,Seafood Restaurant,Historic Site,Ice Cream Shop,Harbor / Marina,Chocolate Shop,Diner,Gift Shop,Clothing Store,Boat or Ferry
8,China Basin,37.776330,-122.391839,2,Baseball Stadium,Coffee Shop,New American Restaurant,Gym / Fitness Center,Baseball Field,Bar,Outdoor Sculpture,Pier,Pizza Place,Athletics & Sports


<font color='blue'>5.3 Check the dataframe.</font>

In [31]:
sf_merged.tail()
#IF the last row have NaN value, just remove them.
#sf_merged=sf_merged[:-1]
#sf_merged.tail()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,Telegraph Hill,37.800785,-122.404091,0,Italian Restaurant,Pizza Place,Cocktail Bar,New American Restaurant,Coffee Shop,Scenic Lookout,Mexican Restaurant,Dive Bar,Café,Chinese Restaurant
81,Tenderloin,37.784249,-122.413993,2,Vietnamese Restaurant,Coffee Shop,Thai Restaurant,Theater,Cocktail Bar,Speakeasy,Burger Joint,Sandwich Place,Art Gallery,Hotel Bar
84,Union Square,37.787936,-122.407517,1,Boutique,Hotel,Jewelry Store,Men's Store,Clothing Store,Theater,Optical Shop,Cosmetics Shop,Coffee Shop,Café
87,Vista del Mar,37.779281,-122.419236,1,Cocktail Bar,Café,Beer Bar,Theater,Performing Arts Venue,Coffee Shop,French Restaurant,Optical Shop,Juice Bar,Sushi Restaurant
89,Western Addition,37.779559,-122.429810,3,Grocery Store,Boutique,Liquor Store,French Restaurant,Pizza Place,Seafood Restaurant,Park,Theater,Café,Sandwich Place


<font color='blue'>5.4 Visualize the resulting clusters.</font>

In [32]:
# create map
map_clusters = folium.Map(location=[latitude_s, longitude_s], zoom_start=12.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 6. Examine Clusters

<font color='blue'>6.1 Cluster 1</font>

In [33]:
sf_merged.loc[sf_merged['Cluster Labels'] == 0, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,37.794301,Chinese Restaurant,Bakery,Men's Store,Cocktail Bar,New American Restaurant,Szechuan Restaurant,Restaurant,Tea Room,Dim Sum Restaurant
36,37.795147,Bakery,Italian Restaurant,Coffee Shop,Cocktail Bar,Hotel,Park,Tea Room,Dive Bar,French Restaurant
43,37.786804,Italian Restaurant,Cosmetics Shop,American Restaurant,New American Restaurant,Furniture / Home Store,Grocery Store,Gourmet Shop,Men's Store,Miscellaneous Shop
54,37.793262,Hotel,Café,Coffee Shop,Bar,Wine Bar,French Restaurant,Yoga Studio,Gym,Cocktail Bar
56,37.801175,Pizza Place,Bakery,Dive Bar,Coffee Shop,Cocktail Bar,Chinese Restaurant,Park,Scenic Lookout,Café
68,37.788751,Playground,American Restaurant,Miscellaneous Shop,Baseball Field,Salon / Barbershop,Park,Men's Store,Gourmet Shop,Italian Restaurant
71,37.800073,Park,Sushi Restaurant,Coffee Shop,Garden,Café,Playground,Pizza Place,Diner,Dive Bar
80,37.800785,Pizza Place,Cocktail Bar,New American Restaurant,Coffee Shop,Scenic Lookout,Mexican Restaurant,Dive Bar,Café,Chinese Restaurant


<font color='blue'>6.2 Cluster 2</font>

In [34]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37.776360,Hotel,Liquor Store,Seafood Restaurant,Café,Sushi Restaurant,Record Shop,Pizza Place,Ethiopian Restaurant,Indian Restaurant
6,37.806532,Seafood Restaurant,Historic Site,Ice Cream Shop,Harbor / Marina,Chocolate Shop,Diner,Gift Shop,Clothing Store,Boat or Ferry
10,37.779594,Coffee Shop,Hotel,Cocktail Bar,Beer Bar,Theater,Performing Arts Venue,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dance Studio
11,37.765813,Breakfast Spot,Thai Restaurant,Pizza Place,Shoe Store,Gift Shop,Record Shop,Bookstore,Wine Bar,Boutique
12,37.764886,Gay Bar,Cosmetics Shop,Scenic Lookout,Indie Movie Theater,Bakery,Salon / Barbershop,Food,Dog Run,Thai Restaurant
13,37.797262,French Restaurant,Wine Bar,Italian Restaurant,Gym / Fitness Center,Thai Restaurant,Salad Place,Mexican Restaurant,Juice Bar,Sushi Restaurant
17,37.767138,New American Restaurant,Sushi Restaurant,Grocery Store,Gay Bar,Gym,Sandwich Place,Pet Store,Mexican Restaurant,Jewelry Store
21,37.786195,Ice Cream Shop,Bakery,Japanese Restaurant,Tea Room,Cosmetics Shop,Ramen Restaurant,Boutique,Grocery Store,Jazz Club
24,37.809167,Ice Cream Shop,Historic Site,Breakfast Spot,Hotel,Boat or Ferry,Clothing Store,Gym / Fitness Center,Tour Provider,Food Truck
28,37.770015,Clothing Store,Café,Thrift / Vintage Store,Coffee Shop,Shoe Store,Board Shop,Breakfast Spot,Bookstore,Thai Restaurant


<font color='blue'>6.3 Cluster 3</font>

In [35]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,37.780836,Health & Beauty Service,Bus Stop,Bus Line,Coffee Shop,Tunnel,Grocery Store,Arts & Crafts Store,Big Box Store,Donut Shop
4,37.791744,Gym,Cocktail Bar,Gym / Fitness Center,Café,Sushi Restaurant,Bubble Tea Shop,Food Truck,Men's Store,Boutique
8,37.776330,Coffee Shop,New American Restaurant,Gym / Fitness Center,Baseball Field,Bar,Outdoor Sculpture,Pier,Pizza Place,Athletics & Sports
18,37.792864,Food Truck,Café,Seafood Restaurant,Sandwich Place,Dessert Shop,Japanese Restaurant,Gym,Mediterranean Restaurant,Deli / Bodega
22,37.793647,Food Truck,Sandwich Place,Café,Hotel,Gym / Fitness Center,Gym,Men's Store,Juice Bar,Japanese Restaurant
23,37.793647,Food Truck,Sandwich Place,Café,Hotel,Gym / Fitness Center,Gym,Men's Store,Juice Bar,Japanese Restaurant
44,37.784316,Vietnamese Restaurant,Coffee Shop,Sandwich Place,Mexican Restaurant,Speakeasy,Sushi Restaurant,Bar,Korean Restaurant,Cocktail Bar
45,37.779096,Café,Wine Bar,Furniture / Home Store,Thai Restaurant,Liquor Store,Movie Theater,Mexican Restaurant,Bank,Baseball Field
50,37.770774,Coffee Shop,Gym,Pizza Place,Pharmacy,Café,Farmers Market,Street Food Gathering,Park,Dog Run
62,37.765181,Art Gallery,Park,American Restaurant,Café,Furniture / Home Store,Mexican Restaurant,Bus Station,Gym,Sushi Restaurant


<font color='blue'>6.4 Cluster 4</font>

In [36]:
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
89,37.779559,Boutique,Liquor Store,French Restaurant,Pizza Place,Seafood Restaurant,Park,Theater,Café,Sandwich Place


<font color='blue'>6.5 Cluster 5</font>

In [37]:
sf_merged.loc[sf_merged['Cluster Labels'] == 4, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,37.765267,Wine Bar,Convenience Store,Coffee Shop,Food,Organic Grocery,Bus Station,Park,Monument / Landmark,Shoe Store
